PACKAGES

In [1]:
from deepod.models.tabular import DevNet, PReNet, DeepSAD, FeaWAD, RoSAS
from deepod.metrics import tabular_metrics
# from autoencodernn import *
# from tapnet import *

/home/work/anaconda3/envs/dongjae_rba/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
import os
from datetime import datetime

In [3]:
import numpy as np
import pandas as pd

In [4]:
# from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

In [5]:
from hyperopt import hp, fmin, Trials, tpe, space_eval

DATASET

In [6]:
train_df = pd.read_csv('./checkpoint/train_df.csv', index_col = 0)
valid_df = pd.read_csv('./checkpoint/valid_df.csv', index_col = 0)
test_df = pd.read_csv('./checkpoint/test_df.csv', index_col = 0)

In [7]:
X_train = pd.read_csv('./checkpoint/x_train.csv')
y_train = pd.read_csv('./checkpoint/y_train.csv')

X_valid = pd.read_csv('./checkpoint/x_valid.csv')
y_valid = pd.read_csv('./checkpoint/y_valid.csv')

X_test = pd.read_csv('./checkpoint/x_test.csv')
y_test = pd.read_csv('./checkpoint/y_test.csv')

MODEL

In [8]:
models = []

In [9]:
def anomaly_preprocessing(df1, df2, df3):
    df1['type'] = 'train'
    df2['type'] = 'valid'
    df3['type'] = 'test'

    df = pd.concat([df1, df2, df3]).reset_index(drop = True)

    country_onehot = pd.get_dummies(df['country']).astype(int)
    risk_grades = df[['region_risk_grade', 'city_risk_grade', 'name_risk_grade']]
    browser_onehot = pd.get_dummies(df['browser_name']).astype(int)
    os_onehot = pd.get_dummies(df['os_name']).astype(int)
    legacys = df[['browser_is_legacy', 'os_is_legacy']]
    device_types = pd.get_dummies(df['device_type']).astype(int)
    rtts = df['rtt']
    type = df['type']
    label = df['label']
    df = pd.concat([country_onehot, risk_grades, browser_onehot, os_onehot, legacys, device_types, rtts, type, label], axis = 1)

    df1 = df[df['type'] == 'train'].drop('type', axis = 1)
    df2 = df[df['type'] == 'valid'].drop('type', axis = 1)
    df3 = df[df['type'] == 'test'].drop('type', axis = 1)

    return df1, df2, df3

In [10]:
# Anomaly Detection
print('Anomaly Detection Model')
ad_model_names = ['DevNet', 'PReNet', 'DeepSAD', 'FeaWAD', 'RoSAS']
ad_models = [
    # DevNet(),           # 86.5s
    # PReNet(),           # Too long. (2.4 hours)
    DeepSAD(),          # 75.4s
    # FeaWAD(epochs = 10000, lr = 0.01),           # Very fast but poor.
    # RoSAS(),            # Too long.
]

train_df_ad, valid_df_ad, test_df_ad = anomaly_preprocessing(train_df, valid_df, test_df)

X_train_ad = train_df_ad.drop(['label'], axis = 1)
y_train_ad = train_df_ad['label']

X_valid_ad = valid_df_ad.drop(['label'], axis = 1)
y_valid_ad = valid_df_ad['label']

X_test_ad = test_df_ad.drop(['label'], axis = 1)
y_test_ad = test_df_ad['label']

for model_name, model in zip(ad_model_names, ad_models):
    print('start -', datetime.now())
    
    model.fit(X_train_ad.to_numpy(), y_train_ad.to_numpy())
    print('Train Finish')
    pred_train = (model.decision_function(X_train_ad.to_numpy()) > 0.5).astype(int)
    auc_train, ap_train, f1_train = tabular_metrics(y_train_ad, pred_train)
    
    pred_valid = (model.decision_function(X_valid_ad.to_numpy()) > 0.5).astype(int)
    auc_valid, ap_valid, f1_valid = tabular_metrics(y_valid_ad, pred_valid)
    
    print(f'Trained with {model}')
    print(f'Train - AUC: {auc_train}, AP: {ap_train}, F1: {f1_train}')
    print(f'Valid - AUC: {auc_valid}, AP: {ap_valid}, F1: {f1_valid}')

    models.append(model)
    print('end -', datetime.now(), '\n')

Anomaly Detection Model
start - 2023-12-11 17:29:43.038737
Start Training...
ensemble size: 1
training data counter: Counter({0: 971745, -1: 94036})
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=220, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 34.687329, time: 95.7s
epoch 10, training loss: 1.323776, time: 87.8s
epoch 20, training loss: 0.993548, time: 87.7s
epoch 30, training loss: 0.952602, time: 78.4s
epoch 40, training loss: 0.844039, time: 78.4s
epoch 50, training loss: 0.795631, time: 79.1s
epoch 60, training loss: 0.732082, time: 77.2s
epoch 70, training loss: 0.791136, time: 78.4s
epoch 80, training loss: 0.979130, time: 78.9s
epoch 90, trai

testing: 100%|██████████| 16653/16653 [00:24<00:00, 678.01it/s]


Train Finish


testing: 100%|██████████| 2379/2379 [00:03<00:00, 723.02it/s]


Trained with <deepod.models.tabular.dsad.DeepSAD object at 0x7fb09596d2e0>
Train - AUC: 0.7752313003374378, AP: 0.18231019854375657, F1: 0.31301570028171
Valid - AUC: 0.7748725965818417, AP: 0.18207407279873802, F1: 0.3126856693918637
end - 2023-12-11 19:46:30.562138 



In [12]:
with open('./checkpoint/deepsad2.pkl', 'wb') as f:
    pickle.dump(models, f)

: 

HYPEROPT TUNING

In [ ]:
search_space = {
    'epochs': hp.quniform('epochs', 10, 200, 10),
    'batch_size': hp.quniform('batch_size', 16, 128, 16),
    'lr': hp.uniform('lr', 1e-7, 1e-1),
    'rep_dim': hp.quniform('rep_dim', 64, 256, 16),
}

In [ ]:
def objective(space):
    model = DeepSAD(epochs = int(space['epochs']), batch_size = int(space['batch_size']), lr = float(space['lr']), rep_dim = int(space['rep_dim']), verbose = 0)

    model.fit(X_train_ad.to_numpy(), y_train_ad.to_numpy())
    
    pred_valid = (model.decision_function(X_valid_ad.to_numpy()) > 0.5).astype(int)
    auc_valid, ap_valid, f1_valid = tabular_metrics(y_valid_ad, pred_valid)
    
    return (1 - auc_valid) ** 2 + (1 - ap_valid) ** 2 + (1 - f1_valid) ** 2

In [ ]:
trials = Trials()

In [ ]:
best = fmin(fn = objective, space = search_space, algo = tpe.suggest, max_evals = 20, trials = trials)

In [ ]:
model = DeepSAD(epochs = int(best['epochs']), batch_size = int(best['batch_size']), lr = float(best['lr']), rep_dim = int(best['rep_dim']), verbose = 2)

model.fit(X_train_ad.to_numpy(), y_train_ad.to_numpy())

pred_train = (model.decision_function(X_train_ad.to_numpy()) > 0.5).astype(int)
auc_train, ap_train, f1_train = tabular_metrics(y_train_ad, pred_train)

pred_valid = (model.decision_function(X_valid_ad.to_numpy()) > 0.5).astype(int)
auc_valid, ap_valid, f1_valid = tabular_metrics(y_valid_ad, pred_valid)

pred_test = (model.decision_function(X_test_ad.to_numpy()) > 0.5).astype(int)
auc_test, ap_test, f1_test = tabular_metrics(y_test_ad, pred_test)